In [ ]:
# import Library
!pip install bayesian-optimization
import pandas as pd
import numpy as np
from numpy import argmax
import matplotlib.pyplot as plt
%matplotlib inline
import librosa as lb
import librosa.display
import IPython.display
import random
import warnings
warnings.filterwarnings('ignore')
import os
from PIL import Image
import pathlib
import csv
from sklearn.model_selection import train_test_split
import keras
from keras import layers
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.layers import Activation,Dense,Dropout,Conv2D,Flatten,MaxPooling2D,GlobalMaxPooling2D,GlobalAveragePooling1D,AveragePooling2D,Input,Add,BatchNormalization
from tensorflow.python.keras.optimizer_v2 import rmsprop
from bayes_opt import BayesianOptimization
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import SGD


  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=70af652f985ea76694450937c727bcc390d08b47f7f54a7aa78f6775385fae27
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# convert audio to spectograms
data_audio = 'Covid Normal'.split( )
plt.figure(figsize=(8,8))
for i in data_audio:
  pathlib.Path(f'img_data/{i}').mkdir(parents = True, exist_ok = True)
  for filename in os.listdir(f'/content/drive/MyDrive/Datasets_Suara Batuk/{i}'):
    audio_class = f'/content/drive/MyDrive/Datasets_Suara Batuk/{i}/{filename}'
    y,sr = librosa.load(audio_class,mono = True, duration= 5)
    print(y.shape)
    plt.specgram(y,NFFT=2048, Fs=2, Fc = 0,noverlap=128,cmap='inferno', sides = 'default',mode = 'default',scale = 'dB')
    plt.axis('off')
    plt.savefig(f'img_data/{i}/{filename[:-3].replace("."," ")}.png')
    plt.clf()


(92610,)
(110250,)
(110250,)
(47628,)
(110250,)
(110250,)
(46305,)
(46305,)
(46305,)
(93933,)
(110250,)
(110250,)
(110250,)
(46305,)
(110250,)
(93933,)
(110250,)
(110250,)
(70119,)
(110250,)
(47628,)
(46305,)
(110250,)
(46305,)
(110250,)
(93933,)
(46305,)
(110250,)
(92610,)
(110250,)
(110250,)
(110250,)
(110250,)
(92610,)
(93933,)
(92610,)
(110250,)
(47628,)
(110250,)
(110250,)
(93933,)
(92610,)
(92610,)
(110250,)
(110250,)
(70119,)
(92610,)
(93933,)
(110250,)
(46305,)
(46305,)
(70119,)
(110250,)
(47628,)
(46305,)
(110250,)
(46305,)
(93933,)
(47628,)
(46305,)
(110250,)
(110250,)
(46305,)
(93933,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(92610,)
(110250,)
(47628,)
(70119,)
(46305,)
(95256,)
(95256,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(92610,)
(92610,)
(110250,)
(110250,)
(110250,)
(110250,)
(92610,)
(92610,)
(93933,)
(93933,)
(46305,)
(47628,)
(47628,)
(46305,)
(110250,)
(110250,)
(110250,)
(110250,)
(47628,)
(46305,)
(110250,)
(110

<Figure size 576x576 with 0 Axes>

In [ ]:
# membuat header csv
header = 'filename chroma_stft rmse spectral_centroid zero_crossing_rate'
for i in range(1,21):
  header += f'mfcc{i}'
header += ' label'
header = header.split()

In [ ]:
# ekstraksi fitur
file = open('datasets_batuk.csv','w',newline='')
with file:
  write = csv.writer(file)
  write.writerow(header)
kelas = 'Covid Normal'.split()
for j in kelas:
  for filename in os.listdir(f'/content/drive/MyDrive/Datasets_Suara Batuk/{j}'):
    audio_name = f'/content/drive/MyDrive/Datasets_Suara Batuk/{j}/{filename}'
    y,sr = librosa.load(audio_name, mono=True, duration = 5)
    rmse = librosa.feature.rms(y=y)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    collect = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(zcr)}'
    for g in mfcc:
      collect += f'{np.mean(g)}'
    collect += f' {g}'
    file = open('datasets_batuk.csv','a',newline = '')
    with file:
      write = csv.writer(file)
      write.writerow(collect.split())






In [ ]:
from pandas._libs.hashtable import Float32Vector
import pandas as pd
data = pd.read_csv('datasets_batuk.csv',on_bad_lines = 'skip')
data.head()
# dropping useless kolom
data.drop(['filename'], axis = 1)
audio_list = data.iloc[:, -1]
encoder = LabelEncoder()
data_y = encoder.fit_transform(audio_list)
scaler = StandardScaler()
data_x = 0
try:
  data_x = float(scaler.fit_transform(np.array(data.iloc[:,:-1])))
except:
  print("can't casting to float")
data.head()


can't casting to float


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,filename,chroma_stft,rmse,spectral_centroid,zero_crossing_ratemfcc1mfcc2mfcc3mfcc4mfcc5mfcc6mfcc7mfcc8mfcc9mfcc10mfcc11mfcc12mfcc13mfcc14mfcc15mfcc16mfcc17mfcc18mfcc19mfcc20,label
sample-5~1.wav,0.49983713030815125,0.005757352337241173,4100.462717,0.25342606180939226-614.761291503906235.327743530273448.8772974014282238.20494365692138712.8348665237426764.5395050048828125-0.11469314992427826-1.24277222156524663.098479986190796-1.82665419578552250.65707737207412721.6340681314468384-0.4799707829952240.23214341700077057-4.610812187194824-5.022896766662598-1.1370874643325806-0.6728976368904114-4.1508483886718751.6671125888824463,[,6.45019341e+00,4.36609173e+00,-0.006005,-1.300261,-0.257250,3.943238,1.290708,-3.036669,-1.508810,-2.306886,-5.045107,-2.359034,-0.329108,-0.741834,0.997529,-2.624031,-4.057709,2.127537,4.513091,1.331565,-1.484086,-0.870203,-2.601876,-3.166917,-2.651533,-5.038945,-7.544968,-5.946695,-5.318800,-1.327940,-1.923682,2.626616,0.346278,-0.930762,2.245816,8.653492,12.531477,-5.956555,-11.457722,-12.531862,-9.240168,-7.958621,-9.082844,-6.878037,3.519963,3.422829,6.409034,10.141186,11.053040,12.677730,12.737963,15.046488,19.813683,18.375116,6.641249,-7.627954,-7.613830,-6.095852,-4.426373,1.916453,7.444955,1.866389,3.113205,-2.006074,-0.023136,-0.845157,-2.380669,-0.293967,2.193871,0.639552,4.373400,4.939681,-0.066659,0.120670,-0.555928,-0.835978,2.437097,3.657032,1.397987,-3.918917,-2.905270,1.608261,-2.883730,-4.166924,5.404973,6.795555,4.613164,6.661002,7.768259,3.61785150e+00,1.26509118e+00,5.25341034e+00,5.13994884e+00,3.633343,-2.22106528e+00,-4.71542645e+00,-3.763679,1.347601,2.278771,5.745000,6.678300,1.176279,1.848322,2.001417,-1.372173,0.338208,0.401592,4.676176,13.573545,12.872938,11.585785,11.936223,5.30338,0.441635,2.990505,8.405586,6.563991,2.587320,3.150610,4.572846,1.783764,-1.162394,-4.452169,1.021306,7.348500,8.268567,6.105976,5.818007,4.13560534e+00,4.656700,11.174776,12.468263,6.838377,2.958368,0.649387,6.43165410e-02,-2.96575975e+00,3.87753308e-01,3.33034420e+00,2.564818,0.957982,0.479553,0.516819,3.895301,6.269473,0.490327,0.702401,-0.215843,1.544431,2.835801,0.575724,-3.550562,-1.432781,-7.715474,-4.396943,0.104728,-6.848922,-5.163200,-3.845131,0.030056,0.263885,2.780123,7.827806,8.164518,9.616638,4.988188,4.238276,2.959823,-1.15540,-5.167830,-8.263052,-3.958928,1.288755,4.07344,3.119769,8.983875,9.178262,7.642234,5.894217,1.84025025e+00,-2.19112372e+00]
sample-38.wav,0.5468164086341858,0.003673197701573372,3568.502866,0.2774424451462766-597.873107910156247.40031814575195-45.28456497192383-6.446096420288086-0.10043702274560928-13.9254837036132816.866733074188232-5.387739181518555-15.732770919799805-10.535965919494629-12.511051177978516-6.665649890899658-6.667151451110841.8227975368499756-10.369089126586914-2.143690586090088-5.8707890510559080.31280604004859924-2.7928209304809571.4418007135391235,[,3.3776007,2.862918,5.482423,3.173013,3.580475,4.470986,-1.937994,-0.128682,3.483325,5.477206,10.902334,9.767478,5.417746,1.274589,-4.131053,-0.845780,3.319076,4.969258,9.230165,5.097994,4.658018,7.223595,0.868650,-0.485420,5.294165,3.603632,1.396121,-3.342217,-4.493138,0.471611,-2.537244,-6.246865,-7.177497,-5.469646,1.429521,20.941269,26.836056,19.228370,13.657501,12.493776,9.968504,8.339774,8.790232,11.195906,8.667628,8.380837,1.115143,-4.939330,-2.220569,-2.399182,-6.625050,6.345884,1.600262,-7.853409,-5.739388,-4.582864,-7.986197,-8.671812,-2.689286,-0.515104,-0.817098,1.864081,5.165687,6.453287,2.426956,2.003808,-6.672114,-7.185983,-4.086527,0.398023,1.413623,-0.228349,-2.066023,-5.659963,-6.241763,-5.617785,-2.653876,-1.482988,0.394744,3.448269,2.081341,-3.445359,-3.224075,-6.327810,0.060626,3.696137,-0.051482,-0.565454,-3.729406,-5.359301,-2.1400151,1.3141893,-0.39019728,-0.621270,],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [ ]:
# split data to training
!pip install split_folders
import splitfolders
splitfolders.ratio('/content/img_data',output="./data",seed = 1337,ratio = (.8, .2))


Copying files: 206 files [00:00, 2726.32 files/s]


In [ ]:
from keras_preprocessing.image import ImageDataGenerator
train_data = ImageDataGenerator(
             rescale = 1./255,#range to (0,1)
             shear_range = 0.2, #random transform
             zoom_range = 0.2,
             horizontal_flip = True)
training_set = train_data.flow_from_directory(
              './data/train',target_size =(64,64),
              batch_size=32,class_mode='categorical',
              shuffle = False)
test_set = train_data.flow_from_directory(
                  './data/val',
                  target_size = (64,64),
                  batch_size = 32,
                  class_mode = 'categorical',
                  shuffle = False
              )


Found 164 images belonging to 2 classes.
Found 42 images belonging to 2 classes.


In [ ]:

# Create CNN functions
from tensorflow.keras import optimizers
def get_model(dropout2_rate=0.5, dense_neuron= 128):
# dropout2_rate = 0.5
  model = Sequential()
  input_shape = (64, 64, 3)#layer hidden ke 1
  model.add(Conv2D(32, (3, 3), strides=(2,2),input_shape = input_shape))
  model.add(AveragePooling2D((2,2),strides=(2,2)))
  model.add(Activation('relu'))#layer hidden ke 2
  model.add(Conv2D(dense_neuron,(3,3),padding='same'))
  model.add(AveragePooling2D((2,2),strides=(2,2)))
  model.add(Activation('relu'))#Layer hidden ke 3
  model.add(Conv2D(dense_neuron,(3,3),padding='same'))
  model.add(AveragePooling2D((2,2),strides=(2,2)))
  model.add(Activation('relu'))#Flatten
  model.add(Flatten())
  model.add(Dropout(rate = dropout2_rate))#fully connected layer
  model.add(Dense(64))
  model.add(Activation('relu'))
  model.add(Dropout(rate=dropout2_rate))#output layer
  model.add(Dense(2))
  model.add(Activation('softmax'))
  model.summary()
  return model
# model = get_model(0.5, 128 )
model = get_model( 0.5, 128)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 31, 31, 32)        896       
                                                                 
 average_pooling2d (AverageP  (None, 15, 15, 32)       0         
 ooling2D)                                                       
                                                                 
 activation (Activation)     (None, 15, 15, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 15, 15, 128)       36992     
                                                                 
 average_pooling2d_1 (Averag  (None, 7, 7, 128)        0         
 ePooling2D)                                                     
                                                                 
 activation_1 (Activation)   (None, 7, 7, 128)         0

In [ ]:
import tensorflow as tf

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5)
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("cnn.h5", save_best_only=True)

def fit_model(learning_rate):
  epochs = 50
  batch_size = 32
  # learning_rate = 0.001
  decay_rate = learning_rate/epochs
  momentum = 0.9

  sgd = SGD(lr = learning_rate, momentum=momentum,decay = decay_rate,nesterov=False)
  model.compile(optimizer="sgd",loss = "categorical_crossentropy",metrics=['accuracy'])
  # model.compile(loss=tf.keras.losses.categorical_crossentropy,optimizer= 'adam', metrics=['accuracy'])

  history = model.fit_generator(
  training_set,
  # steps_per_epoch = len(training_set),
  epochs = 150,
  validation_data = test_set,
  callbacks=[ checkpoint_cb]
  )


  akurasi = model.evaluate_generator(generator=test_set)
  print(f"Loss : {akurasi[0]}, Accuracy : {akurasi[1]}")
  return akurasi[1]

  # return history
  # history = model.fit_generator(
  #   training_set,
  #   steps_per_epoch = len(training_set),
  #   epochs = 100,
  #   validation_data = test_set,
  #   validation_steps = 100
  #   )


In [ ]:
optimizer = BayesianOptimization(
    # get_model,
    fit_model,
    {
        'learning_rate':(0.0001, 0.01)

    }
)
optimizer.maximize(n_iter = 5, init_points = 1)
optimizer.max


|   iter    |  target   | learni... |
-------------------------------------
Epoch 1/150
6/6 [==============================] - 5s 695ms/step - loss: 0.7173 - accuracy: 0.5366 - val_loss: 0.6907 - val_accuracy: 0.5476
Epoch 2/150
6/6 [==============================] - 5s 911ms/step - loss: 0.7158 - accuracy: 0.5366 - val_loss: 0.6865 - val_accuracy: 0.5476
Epoch 3/150
6/6 [==============================] - 7s 1s/step - loss: 0.7160 - accuracy: 0.5488 - val_loss: 0.6883 - val_accuracy: 0.5476
Epoch 4/150
6/6 [==============================] - 5s 735ms/step - loss: 0.7113 - accuracy: 0.5366 - val_loss: 0.6884 - val_accuracy: 0.5476
Epoch 5/150
6/6 [==============================] - 3s 528ms/step - loss: 0.7113 - accuracy: 0.5427 - val_loss: 0.6885 - val_accuracy: 0.5476
Epoch 6/150
6/6 [==============================] - 3s 635ms/step - loss: 0.7066 - accuracy: 0.5427 - val_loss: 0.6883 - val_accuracy: 0.5476
Epoch 7/150
6/6 [==============================] - 3s 533ms/step - loss: 0.7141 -

{'params': {'learning_rate': 0.00012244633531473744},
 'target': 0.9047619104385376}

In [ ]:
# learning_rate hasil bayes
lr_result = optimizer.max['params'].get('learning_rate')


sgd = SGD(lr = lr_result, momentum=0.9,nesterov=False)
model.compile(optimizer="sgd",loss = "categorical_crossentropy",metrics=['accuracy'])
# history = model.fit_generator(
#   training_set,
#   # steps_per_epoch = len(training_set),
#   epochs = 800,
#   validation_data = test_set,
#   callbacks=[early_stopping_cb, checkpoint_cb]
# )

In [ ]:
model_1 = fit_model(lr_result)

Epoch 1/150
6/6 [==============================] - 4s 669ms/step - loss: 0.2712 - accuracy: 0.9024 - val_loss: 0.3057 - val_accuracy: 0.9048
Epoch 2/150
6/6 [==============================] - 3s 537ms/step - loss: 0.2728 - accuracy: 0.9085 - val_loss: 0.3161 - val_accuracy: 0.8571
Epoch 3/150
6/6 [==============================] - 3s 545ms/step - loss: 0.2890 - accuracy: 0.8659 - val_loss: 0.2953 - val_accuracy: 0.8810
Epoch 4/150
6/6 [==============================] - 3s 535ms/step - loss: 0.2479 - accuracy: 0.9085 - val_loss: 0.3587 - val_accuracy: 0.8333
Epoch 5/150
6/6 [==============================] - 3s 538ms/step - loss: 0.2295 - accuracy: 0.9146 - val_loss: 0.3603 - val_accuracy: 0.8571
Epoch 6/150
6/6 [==============================] - 3s 541ms/step - loss: 0.2952 - accuracy: 0.8780 - val_loss: 0.3593 - val_accuracy: 0.8571
Epoch 7/150
6/6 [==============================] - 3s 647ms/step - loss: 0.2731 - accuracy: 0.8659 - val_loss: 0.3412 - val_accuracy: 0.8571
Epoch 8/150
6

In [ ]:
akurasi = model.evaluate_generator(generator=test_set)
print(f"Loss : {akurasi[0]}, Accuracy : {akurasi[1]}")


Loss : 0.36514467000961304, Accuracy : 0.8571428656578064


In [ ]:
# tf.keras.callbacks.History()
# acc = history.history['accuracy']
# val_acc = history.history['val_accuracy']
# loss =np.array(history.history['loss'])
# val_loss =np.array(history.history['val_loss'])
# epoch_range = range(0, len(acc))

# print("", epoch_range)
# print(acc)
# print(epoch_range, "\n")
# print(acc,"\n")
# print(val_acc, "\n")
# print(loss,"\n")
# print(val_loss,"\n")

# plt.figure(figsize = (15,10))
# plt.subplot(1,2,1)
# plt.plot(epoch_range, acc, label = "Akurasi Training",color = 'r')
# plt.plot(epoch_range,val_acc, label = "Akurasi Validasi")
# plt.legend(loc = 'lower right')
# plt.title('Akurasi Training ')
# plt.savefig('Akurasi Training.png', dpi = 200)
# plt.figure(figsize = (15,10))
# plt.subplot(1,2,2)
# plt.plot(epoch_range, loss, label = "Training Loss",color='g')
# plt.plot(epoch_range, val_loss, label = "Validasi Loss")
# plt.legend(loc = 'upper right')
# plt.title('Training Loss')
# plt.savefig('Training Loss.png', dpi = 200)
# plt.show()


In [ ]:
# akurasi = model.evaluate_generator(generator=test_set,steps=len(training_set))
# print(f"Loss : {akurasi[0]}, Accuracy : {akurasi[1]}")


In [ ]:
test_set.reset()
pred = model.predict_generator(test_set, verbose=1)

2/2 [==============================] - 1s 126ms/step


In [ ]:
predicted_class_indices = np.argmax(pred,axis = 1)
labels = (training_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
prediction = [labels[k] for k in predicted_class_indices]
prediction = prediction[:200]
filenames = test_set.filenames

print(len(filename), len(prediction))


9 42


In [ ]:
results = pd.DataFrame({
          "Filename":filenames,
          "Predictions":prediction})
results.to_csv("predictions_results.csv",index = False)
results.head(100)

,Filename,Predictions
0,Covid/sample-13 (1) .png,Covid
1,Covid/sample-15 .png,Covid
2,Covid/sample-19 .png,Covid
3,Covid/sample-21 (1) .png,Covid
4,Covid/sample-27 .png,Covid
5,Covid/sample-29 (1) .png,Covid
6,Covid/sample-30 (1) .png,Covid
7,Covid/sample-31 .png,Covid
8,Covid/sample-32 (1) .png,Normal
9,Covid/sample-32 .png,Normal


In [ ]:
import joblib as jb
jb.dump(model, "/content/drive/MyDrive/Papper Covid/Model/Model_CNN.pkl")



INFO:tensorflow:Assets written to: ram://e5347735-24bd-45aa-b7ad-3131c86c1e1c/assets


['/content/drive/MyDrive/Papper Covid/Model/Model_CNN.pkl']

In [ ]:
!pip install pydub
from google.colab import files
from pydub import AudioSegment
import IPython.display as ipd
import numpy as np
from google.colab import files
import soundfile as sf
# import matplotlib.pyplot as plt
# from keras.preprocessing import image
# import matplotlib.image as mpimg
# %matplotlib inline

upload = files.upload()
src_0= "smpl.wav"

for fn in upload.keys():
  path = fn
  aud_seg = AudioSegment.from_mp3(path)
  aud_seg.export(src_0, format = 'wav')
  path , sr = librosa.load(src_0)
sf.write(f'/content/drive/MyDrive/Papper Covid/result_batuk/{src_0}',np.array([42], dtype='int32') ,44100)
ipd.Audio(src_0)


In [ ]:
!pip install pydub
from google.colab import files
from pydub import AudioSegment
import IPython.display as ipd
import numpy as np
from google.colab import files
import matplotlib.pyplot as plt
from keras.preprocessing import image
import matplotlib.image as mpimg
%matplotlib inline

Submit = files.upload()
src_1= "sampel.wav"

for fn in Submit.keys():
  path = fn
  aud_seg = AudioSegment.from_mp3(path)
  aud_seg.export(src_1, format = 'wav')
  path , sr = librosa.load(src_1)
  ipd.Audio(src_1)
  # Waveform
  plt.figure(figsize=(10, 5))
  # Spectogram
  path,sr = librosa.load(src_1,mono = True, duration= 5)
  print(path.shape)
  plt.specgram(path,NFFT=2048, Fs=2, Fc = 0,noverlap=128,cmap='inferno', sides = 'default',mode = 'default',scale = 'dB')
  plt.axis('off')
  plt.savefig(f'/content/drive/MyDrive/Papper Covid/result_batuk/tes {src_0}.png')
  plt.clf()


  X = librosa.stft(path)
  Xdb = librosa.amplitude_to_db(abs(X))
  plt.figure(figsize=(5, 7), dpi = 100)
  librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='log', cmap = 'inferno')
  plt.colorbar()

  # ploting
  img = image.load_img('/content/drive/MyDrive/Papper Covid/result_batuk/tes.png',target_size=(64,64))
  imgplot = plt.imshow(img)
  e = image.img_to_array(img)
  e = np.expand_dims(e,axis = 0)

  images = np.vstack([e])

  test_set.reset()
  classes = model.predict_generator(images)
  if classes.all() == 0:
      print('\nHasil Prediksi : Anda diprediksi POSITIF COVID-19, silahkan melakukan isolasi mandiri !!\nlakukan tes lanjutan untuk memastikan kondisi Anda !\n')
  elif classes.all() == 1 :
      print('\nHasil Prediksi : Anda diprediksi NORMAL (NEGATIF COVID-19)\n')
  else:
      print('MAAF, SISTEM TIDAK DAPAT MENGENALI MASUKKAN ANDA, Silahkan Coba Lagi')

In [ ]:
print(e)